In [1]:
import numpy as np
from osgeo import gdal

In [2]:
def remove_outliers(input_file, output_file):  # 取96%
    # 打开栅格文件
    dataset = gdal.Open(input_file)
    if dataset is None:
        print("无法打开文件: ", input_file)
        return

    # 读取栅格数据
    band = dataset.GetRasterBand(1)
    array = band.ReadAsArray()
    
    # 获取不是NaN的值
    non_nan_values = array[~np.isnan(array)]
    
    # 计算去除前后2%后的百分位数
    non_nan_values = np.sort(non_nan_values)
    num_values = len(non_nan_values)
    low_percentile_index = int(num_values * 0.02)
    high_percentile_index = int(num_values * 0.98)
    low_percentile = non_nan_values[low_percentile_index]
    high_percentile = non_nan_values[high_percentile_index]
    print(input_file, '\n', low_percentile, high_percentile)
    
    
    # 将超出百分位数范围的值设为NaN
    array[array < low_percentile] = np.nan
    array[array > high_percentile] = np.nan

    # 写入处理后的数据到输出文件
    driver = gdal.GetDriverByName('GTiff')
    output_dataset = driver.Create(output_file, dataset.RasterXSize, dataset.RasterYSize, 1, band.DataType)
    output_dataset.SetProjection(dataset.GetProjection())
    output_dataset.SetGeoTransform(dataset.GetGeoTransform())
    output_band = output_dataset.GetRasterBand(1)
    output_band.WriteArray(array)

    # 关闭数据集
    dataset = None
    output_dataset = None

    print("处理完成")

In [3]:
def calculate_average(input_files, output_file):   # 计算均值
    num_files = len(input_files) 
    dataset = gdal.Open(input_files[0])
    if dataset is None:
        print("无法打开文件欸: ", input_files[0])
        return

    # 读取栅格数据
    band = dataset.GetRasterBand(1)
    array_sum = band.ReadAsArray()

    # 统计有效值的数量
    valid_count = np.ones(array_sum.shape)

    for i in range(1, num_files):
        dataset = gdal.Open(input_files[i])
        if dataset is None:
            print("无法打开文件: ", input_files[i])
            continue

        band = dataset.GetRasterBand(1)
        array = band.ReadAsArray()

        # 更新数组总和和有效值的数量
        valid_mask = np.logical_not(np.isnan(array))
        array_sum[valid_mask] += array[valid_mask]
        valid_count += valid_mask

    # 计算平均值
    average_array = np.where(valid_count > 0, array_sum / valid_count, np.nan)

    # 写入平均值到输出文件
    driver = gdal.GetDriverByName('GTiff')
    output_dataset = driver.Create(output_file, dataset.RasterXSize, dataset.RasterYSize, 1, band.DataType)
    #print('1')
    #print(dataset.GetProjection())
    output_dataset.SetProjection(dataset.GetProjection())
    output_dataset.SetGeoTransform(dataset.GetGeoTransform())
    output_band = output_dataset.GetRasterBand(1)
    output_band.WriteArray(average_array)

    # 关闭数据集
    dataset = None
    output_dataset = None

    print("平均值计算完成")

In [7]:
# ‘NoneType‘ object has no attribute ‘GetGeoTransform是路径错了
index_names = [
    'apar','EVI2_type_A','EVI2_type_B','EVI2_type_C','fesc_type_A','fesc_type_B','fesc_type_C',
    'FPAR','GPP_CEDAR','GPP_fluxcom','kNDVI_type_B','kNDVI_type_C','LAI','LUE_CEDAR','LUE_fluxcom','NDVI_type_A',
    'NDVI_type_B','NDVI_type_C','new_kNDVI_type_C','NIRv_type_A','NIRv_type_B','NIRv_type_C','NIRvP','NIRvR',
    'par','sif','SIF_yield_P','SIF_yield_R','sifapar'
]
region = 'cornbelt'
if __name__ == "__main__":
    list_90 = []
    k = 18
    while k <= 20:
        print(k)
        for i in index_names: 
            input_file =  r"C:\Users\levi\cartopy\5.6data\{}\\{}_20{}_{}.tif".format(region,i,k,region)  # 输入栅格文件名
            output_file = r"C:\Users\levi\cartopy\5.6data\ninety\\{}\\{}_20{}_{}_90.tif".format(region,i,k,region)            
            print(input_file)
            list_90.append(output_file)
            remove_outliers(input_file, output_file)
        k += 1
print(list_90)

18
C:\Users\levi\cartopy\5.6data\cornbelt\\apar_2018_cornbelt.tif
C:\Users\levi\cartopy\5.6data\cornbelt\\apar_2018_cornbelt.tif 
 952.09314 1406.3774
处理完成
C:\Users\levi\cartopy\5.6data\cornbelt\\EVI2_type_A_2018_cornbelt.tif
C:\Users\levi\cartopy\5.6data\cornbelt\\EVI2_type_A_2018_cornbelt.tif 
 0.44590166 0.717807
处理完成
C:\Users\levi\cartopy\5.6data\cornbelt\\EVI2_type_B_2018_cornbelt.tif
C:\Users\levi\cartopy\5.6data\cornbelt\\EVI2_type_B_2018_cornbelt.tif 
 0.43611085 0.7186238
处理完成
C:\Users\levi\cartopy\5.6data\cornbelt\\EVI2_type_C_2018_cornbelt.tif
C:\Users\levi\cartopy\5.6data\cornbelt\\EVI2_type_C_2018_cornbelt.tif 
 0.43780652 0.7113629
处理完成
C:\Users\levi\cartopy\5.6data\cornbelt\\fesc_type_A_2018_cornbelt.tif
C:\Users\levi\cartopy\5.6data\cornbelt\\fesc_type_A_2018_cornbelt.tif 
 0.36535874009132385 0.6024211049079895
处理完成
C:\Users\levi\cartopy\5.6data\cornbelt\\fesc_type_B_2018_cornbelt.tif
C:\Users\levi\cartopy\5.6data\cornbelt\\fesc_type_B_2018_cornbelt.tif 
 0.35670202970

C:\Users\levi\cartopy\5.6data\cornbelt\\sifapar_2019_cornbelt.tif 
 0.00040038934 0.00080240105
处理完成
20
C:\Users\levi\cartopy\5.6data\cornbelt\\apar_2020_cornbelt.tif
C:\Users\levi\cartopy\5.6data\cornbelt\\apar_2020_cornbelt.tif 
 1016.0635 1548.3413
处理完成
C:\Users\levi\cartopy\5.6data\cornbelt\\EVI2_type_A_2020_cornbelt.tif
C:\Users\levi\cartopy\5.6data\cornbelt\\EVI2_type_A_2020_cornbelt.tif 
 0.40992454 0.69627947
处理完成
C:\Users\levi\cartopy\5.6data\cornbelt\\EVI2_type_B_2020_cornbelt.tif
C:\Users\levi\cartopy\5.6data\cornbelt\\EVI2_type_B_2020_cornbelt.tif 
 0.40063226 0.7058447
处理完成
C:\Users\levi\cartopy\5.6data\cornbelt\\EVI2_type_C_2020_cornbelt.tif
C:\Users\levi\cartopy\5.6data\cornbelt\\EVI2_type_C_2020_cornbelt.tif 
 0.402656 0.7002773
处理完成
C:\Users\levi\cartopy\5.6data\cornbelt\\fesc_type_A_2020_cornbelt.tif
C:\Users\levi\cartopy\5.6data\cornbelt\\fesc_type_A_2020_cornbelt.tif 
 0.33749017119407654 0.5132808685302734
处理完成
C:\Users\levi\cartopy\5.6data\cornbelt\\fesc_type_B_20

In [8]:
for i in index_names: 
    input_files = []
    for j in range(2018,2021):
        #print(i,j)
        file_name = r"C:\Users\levi\cartopy\5.6data\ninety\\{}\\{}_{}_{}_90.tif".format(region,i,j,region)
        #print(file_name)
        input_files.append(file_name)
    # print(input_files)
    output_file = r"C:\Users\levi\cartopy\5.6data\averg\\avg_{}_{}.tif".format(i,region)  # 输出栅格文件名
    print(output_file)
    calculate_average(input_files, output_file)

C:\Users\levi\cartopy\5.6data\averg\\avg_apar_cornbelt.tif
平均值计算完成
C:\Users\levi\cartopy\5.6data\averg\\avg_EVI2_type_A_cornbelt.tif
平均值计算完成
C:\Users\levi\cartopy\5.6data\averg\\avg_EVI2_type_B_cornbelt.tif
平均值计算完成
C:\Users\levi\cartopy\5.6data\averg\\avg_EVI2_type_C_cornbelt.tif
平均值计算完成
C:\Users\levi\cartopy\5.6data\averg\\avg_fesc_type_A_cornbelt.tif
平均值计算完成
C:\Users\levi\cartopy\5.6data\averg\\avg_fesc_type_B_cornbelt.tif
平均值计算完成
C:\Users\levi\cartopy\5.6data\averg\\avg_fesc_type_C_cornbelt.tif
平均值计算完成
C:\Users\levi\cartopy\5.6data\averg\\avg_FPAR_cornbelt.tif
平均值计算完成
C:\Users\levi\cartopy\5.6data\averg\\avg_GPP_CEDAR_cornbelt.tif
平均值计算完成
C:\Users\levi\cartopy\5.6data\averg\\avg_GPP_fluxcom_cornbelt.tif
平均值计算完成
C:\Users\levi\cartopy\5.6data\averg\\avg_kNDVI_type_B_cornbelt.tif
平均值计算完成
C:\Users\levi\cartopy\5.6data\averg\\avg_kNDVI_type_C_cornbelt.tif
平均值计算完成
C:\Users\levi\cartopy\5.6data\averg\\avg_LAI_cornbelt.tif
平均值计算完成
C:\Users\levi\cartopy\5.6data\averg\\avg_LUE_CEDAR_cornbelt.